In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import warnings
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn import metrics
from win32timezone import now
import gc
%matplotlib inline

warnings.filterwarnings("ignore")

In [2]:
def feat_select(train, test):
    features = train.drop(['is_trade', 'time', 'context_timestamp'], axis=1).columns.tolist()
    feature = []
    target = ['is_trade']

    return features, target


def xgbCV(train, test, features, target):
    X = train[features]
    y = train[target]
    X_tes = test[features]
    y_tes = test[target]

    print('Training XGB model...')
    X_train_set = xgb.DMatrix(X, label=y, missing=np.nan)
    X_validate_set = xgb.DMatrix(X_tes, label=y_tes, missing=np.nan)
    watchlist = [(X_train_set, 'train'), (X_validate_set, 'eval')]
    params = {'max_depth': 7,
              'nthread': -1,
              'eta': 0.01,
              'eval_metric': 'logloss',
              'objective': 'binary:logistic',
#               'objective':'rank:pairwise',
              'subsample': 0.85,
              'colsample_bytree': 0.85,
              'silent': 1,
              'seed': 0,
              'min_child_weight': 6,
              'gpu_id': 0,
              'tree_method': 'gpu_hist'
#               'max_delta_step':1,
#               'scale_pos_weight':0.5
              }
    history = xgb.cv(params, X_train_set, num_boost_round=3000, nfold=5, early_stopping_rounds=50,verbose_eval=True)
    return history


def sub(train, test, features, target, best_iter_num):
    X = train[features]
    y = train[target]
    X_train_set = xgb.DMatrix(X, label=y, missing=np.nan)
    X_test_set = xgb.DMatrix(test[features], missing=np.nan)
    print('Training XGB model...')
    params = {'max_depth': 7,
              'nthread': -1,
              'eta': 0.01,
              'eval_metric': 'logloss',
              'objective': 'binary:logistic',
#               'objective':'rank:pairwise',
              'subsample': 0.85,
              'colsample_bytree': 0.85,
              'silent': 1,
              'seed': 0,
              'min_child_weight': 6,
#               'gpu_id': 0,
#               'tree_method': 'gpu_hist'
#               'scale_pos_weight':0.5
              }
    bst = xgb.train(params, X_train_set, num_boost_round=best_iter_num)
    bst.save_model('001.model')
    return bst


def train():
    # train_data = rebalance()
    print("starting CV:", now())
    history = xgbCV(train_data, test_data, features, target)
    best_iter=history.shape[0]
    print('best_iter:%s' % best_iter)
#     best_iter=1075
    print('最佳迭代次数：', best_iter)
    return sub(train_data, test_data, features, target, best_iter)


def rebalance():
    sm = SMOTEENN()
    train_data.replace(to_replace=np.nan, value=0, inplace=True)
    train_data.replace(to_replace=-np.inf, value=0, inplace=True)
    train_data.replace(to_replace=np.inf, value=0, inplace=True)
    print("rebalance data:", now())
    X_resampled, y_resampled = sm.fit_resample(train_data[features], train_data[target])
    X_resampled = pd.DataFrame(X_resampled, columns=features)
    y_resampled = pd.DataFrame(y_resampled, columns=target)
    X_resampled['is_trade'] = y_resampled['is_trade']
    del y_resampled
    gc.collect()
    return X_resampled


def count():
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for i in range(0, len(predictions)):
        if predictions[i] == y_true.T[0][i]:
            if predictions[i] == 1:
                tp += 1
            else:
                tn += 1
        else:
            if predictions[i] == 1:
                fp += 1
            else:
                fn += 1

    print(tp, tn, fp, fn)


In [3]:
path = './data/'

data = pd.read_csv(path + 'all_final_data_11-07-18-51.csv')
data = data[data.is_trade.notnull()]
train_data, test_data = train_test_split(data, test_size=0.25)
features, target = feat_select(train_data, test_data)

In [4]:
# print("starting CV:", now())
# history = xgbCV(train_data, test_data, features, target)
# best_iter=history.shape[0]
# history

In [5]:
bst=train()
gc.collect()
bst = xgb.Booster({'nthread': 6})  # init model
bst.load_model('001.model')  # load data

X_test_set = xgb.DMatrix(test_data[features], missing=np.nan)
print('predicting……')
y_pred = bst.predict(X_test_set)
predictions = [round(x) for x in y_pred]
y_true = test_data[target].values
count()
# 输出模型的一些结果
print("\n关于现在这个模型")
print("准确率 : %.4g" % metrics.accuracy_score(y_true, predictions))
print("召回率:%.4f" % metrics.recall_score(y_true, predictions))
print("精度:%.4f" % metrics.precision_score(y_true, predictions))
print("F1:%.4f" % metrics.f1_score(y_true, predictions))
print("AUC 得分 (训练集): %f" % 0.906732)
print("AUC 得分 (测试集): %f" % 0.769061)
# xgb.plot_importance(alg.get_booster())

starting CV: 2019-01-27 17:06:20.057384+08:00
Training XGB model...
[0]	train-logloss:0.683928+4.36348e-06	test-logloss:0.683931+4.49889e-06
[1]	train-logloss:0.674894+6.9685e-06	test-logloss:0.674899+1.00399e-05
[2]	train-logloss:0.666037+1.24193e-05	test-logloss:0.666045+1.49987e-05
[3]	train-logloss:0.657349+1.61047e-05	test-logloss:0.65736+1.95305e-05
[4]	train-logloss:0.64883+1.75545e-05	test-logloss:0.648844+2.598e-05
[5]	train-logloss:0.640473+2.06824e-05	test-logloss:0.64049+3.18283e-05
[6]	train-logloss:0.632276+2.38881e-05	test-logloss:0.632296+3.6788e-05
[7]	train-logloss:0.624233+2.70215e-05	test-logloss:0.624255+4.21303e-05
[8]	train-logloss:0.616337+2.81894e-05	test-logloss:0.616363+4.85946e-05
[9]	train-logloss:0.608588+3.10548e-05	test-logloss:0.608615+5.39459e-05
[10]	train-logloss:0.600982+3.53236e-05	test-logloss:0.601011+5.86195e-05
[11]	train-logloss:0.593513+3.75372e-05	test-logloss:0.593545+6.36943e-05
[12]	train-logloss:0.58618+3.9973e-05	test-logloss:0.586216+6

[111]	train-logloss:0.220802+0.000205072	test-logloss:0.221248+0.000513912
[112]	train-logloss:0.219099+0.000205449	test-logloss:0.21955+0.000518558
[113]	train-logloss:0.217418+0.000207463	test-logloss:0.217873+0.000521685
[114]	train-logloss:0.215757+0.000208023	test-logloss:0.216216+0.000525107
[115]	train-logloss:0.214118+0.000210305	test-logloss:0.214582+0.000528622
[116]	train-logloss:0.2125+0.000211306	test-logloss:0.21297+0.000533228
[117]	train-logloss:0.210903+0.000212182	test-logloss:0.211378+0.000536864
[118]	train-logloss:0.209325+0.000213993	test-logloss:0.209806+0.000540083
[119]	train-logloss:0.207767+0.000214605	test-logloss:0.208253+0.000544792
[120]	train-logloss:0.206229+0.000215892	test-logloss:0.20672+0.000548119
[121]	train-logloss:0.204711+0.000217019	test-logloss:0.205208+0.000552001
[122]	train-logloss:0.203212+0.000218885	test-logloss:0.203714+0.000554863
[123]	train-logloss:0.20173+0.000220502	test-logloss:0.202238+0.000558747
[124]	train-logloss:0.200267+0.

[221]	train-logloss:0.118196+0.000300645	test-logloss:0.119443+0.00089983
[222]	train-logloss:0.117758+0.000301478	test-logloss:0.119014+0.000902907
[223]	train-logloss:0.117325+0.00030156	test-logloss:0.11859+0.000906585
[224]	train-logloss:0.116898+0.000301587	test-logloss:0.118172+0.00090966
[225]	train-logloss:0.116475+0.000302566	test-logloss:0.11776+0.000912904
[226]	train-logloss:0.116056+0.000302018	test-logloss:0.117351+0.000916678
[227]	train-logloss:0.115643+0.000303247	test-logloss:0.116947+0.000920474
[228]	train-logloss:0.115234+0.000305213	test-logloss:0.11655+0.000924474
[229]	train-logloss:0.114831+0.000305462	test-logloss:0.116157+0.000927865
[230]	train-logloss:0.114431+0.000306153	test-logloss:0.115768+0.000931175
[231]	train-logloss:0.114037+0.000305615	test-logloss:0.115386+0.000935487
[232]	train-logloss:0.113648+0.000305839	test-logloss:0.115008+0.000939528
[233]	train-logloss:0.113264+0.000307476	test-logloss:0.114634+0.000941709
[234]	train-logloss:0.112884+0.

[331]	train-logloss:0.0911318+0.00035207	test-logloss:0.0938744+0.0012077
[332]	train-logloss:0.0910118+0.000352305	test-logloss:0.0937702+0.00121115
[333]	train-logloss:0.090893+0.000352601	test-logloss:0.093666+0.00121394
[334]	train-logloss:0.0907748+0.000354507	test-logloss:0.093566+0.00121578
[335]	train-logloss:0.0906576+0.000356696	test-logloss:0.0934648+0.00121793
[336]	train-logloss:0.0905422+0.000356103	test-logloss:0.093366+0.00122014
[337]	train-logloss:0.0904268+0.000357075	test-logloss:0.093269+0.00122171
[338]	train-logloss:0.090313+0.000357494	test-logloss:0.0931732+0.0012226
[339]	train-logloss:0.0902008+0.000357396	test-logloss:0.0930802+0.00122575
[340]	train-logloss:0.0900914+0.000357095	test-logloss:0.0929872+0.00122829
[341]	train-logloss:0.0899796+0.000356718	test-logloss:0.0928944+0.00123021
[342]	train-logloss:0.0898714+0.00035669	test-logloss:0.0928024+0.00123208
[343]	train-logloss:0.0897656+0.000356369	test-logloss:0.0927132+0.00123368
[344]	train-logloss:0.

[440]	train-logloss:0.0830638+0.000373036	test-logloss:0.0877828+0.00142209
[441]	train-logloss:0.0830198+0.000374008	test-logloss:0.0877568+0.00142178
[442]	train-logloss:0.0829756+0.000375595	test-logloss:0.0877314+0.00142431
[443]	train-logloss:0.0829312+0.000375577	test-logloss:0.0877068+0.00142621
[444]	train-logloss:0.0828894+0.000374335	test-logloss:0.0876836+0.00142748
[445]	train-logloss:0.082847+0.000372088	test-logloss:0.0876608+0.00142748
[446]	train-logloss:0.0828062+0.000371512	test-logloss:0.0876378+0.00142906
[447]	train-logloss:0.0827654+0.000371495	test-logloss:0.0876144+0.00143191
[448]	train-logloss:0.0827226+0.000374063	test-logloss:0.0875914+0.00143381
[449]	train-logloss:0.0826784+0.000374149	test-logloss:0.0875692+0.00143571
[450]	train-logloss:0.0826364+0.000376133	test-logloss:0.0875446+0.00143729
[451]	train-logloss:0.0825946+0.000378114	test-logloss:0.087521+0.00144045
[452]	train-logloss:0.0825546+0.000378424	test-logloss:0.0874994+0.00144203
[453]	train-lo

...

[767]	train-logloss:0.0757812+0.000420771	test-logloss:0.0854958+0.00160932
[768]	train-logloss:0.0757692+0.000418119	test-logloss:0.0854946+0.00160965
[769]	train-logloss:0.0757548+0.00042021	test-logloss:0.0854934+0.00160941
[770]	train-logloss:0.0757392+0.000419215	test-logloss:0.085492+0.00160914
[771]	train-logloss:0.0757254+0.000418707	test-logloss:0.0854926+0.00160981
[772]	train-logloss:0.0757138+0.000418806	test-logloss:0.0854908+0.00160983
[773]	train-logloss:0.0757008+0.000422199	test-logloss:0.085491+0.00161084
[774]	train-logloss:0.0756886+0.000417973	test-logloss:0.0854898+0.00161119
[775]	train-logloss:0.0756734+0.000418399	test-logloss:0.0854878+0.00161086
[776]	train-logloss:0.07566+0.000416876	test-logloss:0.085486+0.00161016
[777]	train-logloss:0.0756478+0.000416944	test-logloss:0.0854848+0.00161074
[778]	train-logloss:0.0756354+0.000416061	test-logloss:0.0854834+0.00161046
[779]	train-logloss:0.075622+0.000413962	test-logloss:0.0854808+0.00161012
[780]	train-logloss

[876]	train-logloss:0.074366+0.000425456	test-logloss:0.0853956+0.00163837
[877]	train-logloss:0.0743538+0.000427827	test-logloss:0.0853942+0.00163836
[878]	train-logloss:0.0743394+0.000425928	test-logloss:0.0853934+0.00163837
[879]	train-logloss:0.074329+0.000429348	test-logloss:0.0853922+0.00163802
[880]	train-logloss:0.0743176+0.000432897	test-logloss:0.085392+0.0016383
[881]	train-logloss:0.0743022+0.000432675	test-logloss:0.085391+0.00163772
[882]	train-logloss:0.0742916+0.000434279	test-logloss:0.085391+0.00163839
[883]	train-logloss:0.0742778+0.000438136	test-logloss:0.0853904+0.00163873
[884]	train-logloss:0.0742676+0.000437348	test-logloss:0.0853894+0.00163876
[885]	train-logloss:0.0742544+0.000438398	test-logloss:0.085387+0.00164002
[886]	train-logloss:0.074239+0.000438691	test-logloss:0.0853864+0.00164001
[887]	train-logloss:0.0742282+0.00043704	test-logloss:0.0853862+0.0016407
[888]	train-logloss:0.0742162+0.000439612	test-logloss:0.0853848+0.0016394
[889]	train-logloss:0.0

[985]	train-logloss:0.0730348+0.000451331	test-logloss:0.0853432+0.00165893
[986]	train-logloss:0.073022+0.000449835	test-logloss:0.085343+0.00165954
[987]	train-logloss:0.0730104+0.000449765	test-logloss:0.0853418+0.00166023
[988]	train-logloss:0.0729972+0.000453733	test-logloss:0.0853416+0.00165997
[989]	train-logloss:0.0729874+0.00045398	test-logloss:0.085341+0.00166
[990]	train-logloss:0.072977+0.000453517	test-logloss:0.0853416+0.00166025
[991]	train-logloss:0.0729636+0.000450214	test-logloss:0.085341+0.00166032
[992]	train-logloss:0.072952+0.00044967	test-logloss:0.0853406+0.00165969
[993]	train-logloss:0.072942+0.000446724	test-logloss:0.085341+0.00165935
[994]	train-logloss:0.0729298+0.000446323	test-logloss:0.0853414+0.00165937
[995]	train-logloss:0.0729206+0.000442531	test-logloss:0.085341+0.00165903
[996]	train-logloss:0.0729102+0.000444703	test-logloss:0.0853404+0.00165927
[997]	train-logloss:0.0728962+0.000445216	test-logloss:0.0853398+0.00165957
[998]	train-logloss:0.0728

[1093]	train-logloss:0.0718668+0.000415976	test-logloss:0.0853262+0.00166744
[1094]	train-logloss:0.0718572+0.000415947	test-logloss:0.0853262+0.00166614
[1095]	train-logloss:0.0718494+0.000415335	test-logloss:0.0853266+0.00166617
[1096]	train-logloss:0.071839+0.000414289	test-logloss:0.0853262+0.00166622
[1097]	train-logloss:0.071827+0.000412121	test-logloss:0.085327+0.0016668
[1098]	train-logloss:0.0718168+0.000409394	test-logloss:0.0853272+0.00166581
[1099]	train-logloss:0.0718046+0.000410406	test-logloss:0.0853264+0.00166554
[1100]	train-logloss:0.0717948+0.000411874	test-logloss:0.0853264+0.00166523
[1101]	train-logloss:0.0717832+0.000407795	test-logloss:0.0853272+0.00166553
[1102]	train-logloss:0.0717738+0.000410125	test-logloss:0.085327+0.00166551
[1103]	train-logloss:0.0717618+0.000408237	test-logloss:0.0853268+0.0016652
[1104]	train-logloss:0.0717504+0.000406608	test-logloss:0.0853272+0.00166479
[1105]	train-logloss:0.0717408+0.000404346	test-logloss:0.0853272+0.00166618
[1106

[1201]	train-logloss:0.0707638+0.000394241	test-logloss:0.0853204+0.00168064
[1202]	train-logloss:0.0707522+0.000393663	test-logloss:0.0853204+0.00168127
[1203]	train-logloss:0.0707444+0.000392783	test-logloss:0.0853198+0.00168125
[1204]	train-logloss:0.070734+0.000391382	test-logloss:0.0853214+0.00168282
[1205]	train-logloss:0.0707232+0.000390667	test-logloss:0.0853224+0.00168284
[1206]	train-logloss:0.0707072+0.000391467	test-logloss:0.0853224+0.00168284
[1207]	train-logloss:0.070697+0.000392062	test-logloss:0.0853212+0.00168254
[1208]	train-logloss:0.0706856+0.000388166	test-logloss:0.085322+0.00168282
[1209]	train-logloss:0.0706732+0.000387782	test-logloss:0.085322+0.00168221
[1210]	train-logloss:0.0706632+0.000387276	test-logloss:0.0853208+0.00168188
[1211]	train-logloss:0.0706556+0.000384892	test-logloss:0.0853214+0.00168252
[1212]	train-logloss:0.0706444+0.000385883	test-logloss:0.085322+0.00168227
[1213]	train-logloss:0.0706364+0.000386172	test-logloss:0.0853218+0.00168258
[121

In [6]:
importance = bst.get_fscore()
for key,value in importance.items():
    print(key,value)

item_price_level_mean_by_category 298
user_id_query_day_hour_item_id 229
u_day_diffTime_last 951
item_price_level_mean_by_brand 588
item_pv_level_user_occ_prob 102
item_brand_id_user_age_prob 394
context_id 1301
check_item_category_1_min_diff_next 1564
user_age_level 314
b_day_diffTime_first 546
item_sales_level_mean_by_city_ratio 710
item_pv_level_mean_by_brand_ratio 350
item_sales_level_mean_by_category_ratio 736
item_pv_level_price_prob 376
category_day_active_item 426
item_collected_level 51
user_id_query_day_hour_map_item_id 129
shop_score_delivery 1122
item_pv_level_salse_prob 596
item_price_level_mean_by_city 299
check_shop_id_time_day 92
item_collected_level_salse_prob 890
check_item_price_level_min_diff_next 989
shop_star_level_user_gender_prob 183
item_id_lasttime_diff 1279
click_user_shop_lab 53
item_price_level_mean_by_brand_ratio 417
click_user_item_lab 679
brand_mean_hour 470
item_pv_level_shop_rev_prob 286
i_day_diffTime_last 1359
item_sales_level_shop_rev_prob 540
item_